# Project 3 - Scrap listings in Ebay

**This is a rather complete project where I scrape both sponsored and nonsponsord items information,  then dump all the information in local mysql database.**

The overview of the steps are:  
- get familiar with eBay listing url structure and display norm
- identify "sponsored" and "non-sponsored" items
- get links of the first 10 page of all listings (100 items displayed in each page)
- save the website file
- select information and store them in database in a structured way
- clean the data for further usage

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import numpy as np
import time
import re
import os
from os.path import join as pjoin
import pandas as pd

In [2]:
# the price in pop-up needs to be scraped by selenium
# initiate the solemium package and download the driver from internet
from selenium.webdriver import Chrome
browser = Chrome(executable_path="C:/Users/86136/Downloads/chromedriver_win32/chromedriver.exe")

**a) get familiar with the eBay website url structure:**  

How would you modify your search query to only include buy-it-now (non-auction) items? What is the GET request's variable name corresponding to buy-it-now searches? How would you modify your search query to include 100 items per search result page?
  
**[Answer]** We can modify it by typing different url. For url with all items result, the variable name is LH_ALL =1, for the **buy-it-now** items, the parameter becomes "LH_BIN" and value equals to 1.   

If we want to adjust the display item number, we can modify the parameter "_ipg". Here,we can add &_ipg=100 so that the display item number becomes 100.   
(The search query url is : https://www.ebay.com/sch/i.html?_fsrp=1&_nkw=samsung+tv&from=R40&_ipg=100&rt=nc&LH_BIN=1 )  



  
**b) Modify previous code to search for buy-it-now listings of "playstation 4 slim" and limit the number of items to 100 per page.**
  
Identify all the sponsored items.
  
For the first 10 pages of 100 items/page, save all the URLs of sponsored items' pages to the file "sponsored.txt"  and all the URLs of non-sponsored items' pages to the file "non-sponsored.txt" in the same directory as your code. (One URL per line in each file)

In [ ]:
#Define the url
url = 'https://www.ebay.com/sch/i.html?_from=R40&_nkw=playstation+4+slim&LH_All=1&rt=nc&_ipg=100'
#Request to go to the url
url_obj = requests.get(url)
#Use beautifulsoup to parse the text of the website
soup = BeautifulSoup(url_obj.text, 'html.parser')

In [ ]:
#Define the page pattern of the URL
page_pattern = '&_pgn='
sponsored = ""
non_sponsored = ""

#Iterate the page number by using for loop
for i in range(1,11):
    #10-second pause each time it starts the for loop
    time.sleep(10)
    #get the current page number url
    url = 'https://www.ebay.com/sch/i.html?_from=R40&_nkw=playstation+4+slim&LH_All=1&rt=nc&_ipg=100' + page_pattern + str(i)
    #request to go to the current url
    url_obj = requests.get(url)
    #parse the text of the current url website
    soup = BeautifulSoup(url_obj.text, 'html.parser')
    
    for item in soup.find_all('div', attrs = {'style':'display:flex;'}):
       #Under the div named style:display:flex, use the regex to find the following pattern below
        pattern = re.compile(r'.*[S].*[P].*[O].*[N].*[S].*[O].*[R].*[E].*[D].*')
       #Get the text content of the div class which has the pattern above
        text = item.getText()
        #Search the url link of the sponsored item under div class
        if pattern.search(text):
            #Print the url of the sponsored item which is under href
            sponsored = sponsored + item.parent.get('href') + "\n"
        else:
            non_sponsored = non_sponsored + item.parent.get('href') + "\n"

In [ ]:
with open('sponsored.txt', 'w') as file:
    file.write(sponsored)

with open('non_sponsored.txt', 'w') as file:
    file.write(non_sponsored)

**c) Create two folders in the same directory as your code and name them "sponsored" and "non-sponsored".**  
Write a program that opens the two files in (b) and downloads each of the pages (URLs) into the folders "sponsored" and "non-sponsored". Each file should be named as "<item-id>.htm" where you replace "item-id" with the ID of the item you are saving. E.g., "264616053293.htm" for the item with ID "264616053293". Note it is always good to put a 2-second pause between queries. Make sure to catch an error and continue if your query runs into problems connecting to eBay (e.g., if your internet is down for 5 seconds, you don't want your entire code to crash).

In [ ]:
# Create directory 'sponsored'
dirName = 'sponsored'
 
try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

In [ ]:
# Create directory 'non-sponsored'
dirName = 'non-sponsored'

try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

In [3]:
# Input sponsored item info in folder "sponsored" by different htm file

sponsored_id = [ ]

user_agent = {'User-agent': 'Mozilla/5.0'}

with open('sponsored.txt', 'r',encoding="utf8") as file:
    for line in file:
        try:
            
        #10-second pause each time it starts the for loop
            time.sleep(2)
        #get the current page number url
            url = line 
        #request to go to the current url
            url_obj = requests.get(url)
        #parse the text of the current url website
            soup = BeautifulSoup(url_obj.text, 'html.parser')
        #get item_id
            item = soup.find("div", attrs = {"id": "descItemNumber"})
            if item : 
                itemid =soup.find("div", attrs = {"id": "descItemNumber"}).get_text()
                sponsored_id.append(itemid)
                #write file to local directory
                with open("D:\DataAnaly\python\BAX422\midterm\sponsored\%s.htm" %(itemid),'w',encoding='utf8') as file:
                    file.write(str(soup))
            else: 
                pass
        
        except ConnectionError as e:
            print(e)


In [7]:
# Input non-sponsored item info in folder "sponsored" by different htm file

nonsponsored_id = [ ]

user_agent = {'User-agent': 'Mozilla/5.0'} 

with open('non_sponsored.txt', 'r',encoding="utf8") as file:
    for line in file:
        try:
            
        #10-second pause each time it starts the for loop
            time.sleep(2)
        #get the current page number url
            url = line 
        #request to go to the current url
            url_obj = requests.get(url, headers = user_agent)
        #parse the text of the current url website
            soup = BeautifulSoup(url_obj.text, 'html.parser')
        #get item_id
            item =soup.find("div", attrs = {"id": "descItemNumber"})
            if item:
                itemid = soup.find("div", attrs = {"id": "descItemNumber"}).get_text()
                filename = "%s.htm"%(itemid)
                path_to_file = pjoin("D:\\", "DataAnaly", "python", "BAX422","midterm","non-sponsored", filename)
                nonsponsored_id.append(itemid)
        #write to file
                with open(path_to_file,'w',encoding='utf8') as file:
                    file.write(str(soup))
            else:
                pass
        
        except ConnectionError as e:
            print(e)

**d) Write a separate piece of code that loops through the pages you downloaded in (c) and opens and parses them into a Python or Java xxxxsoup-object. Identify and select:**
  
seller name, seller score, item price, # items sold, best offer available, title, returns allowed, shipping price, condition (e.g., used, new, like new, seller refurbished, ...).
  
In your code, highlight the selector command you choose to obtain each element using comments.

In [83]:
df =  pd.DataFrame(columns=["item_id","item_title","seller_name",
                           "seller_score","item_price","item_sold"
                           ,"best_offer","return_policy"
                           ,"shipping_price","condition","sponsor"])
df

,item_id,item_title,seller_name,seller_score,item_price,item_sold,best_offer,return_policy,shipping_price,condition,sponsor


In [5]:
# define the function for running selenium package to fetch the price in "see the detail" pop-up
def seleniumgetprice(path):
    browser.get(path)
    label = browser.find_element_by_id("finalPrc")
    soup_level2=BeautifulSoup(browser.page_source, 'html.parser')
    price = soup_level2.find('div',attrs={"id":"finalPrc"}).text
    return price

In [84]:
# write all the information of sponsored item in dataframe "df"

for i in sponsored_id:

        
    filename = "%s.htm" % (i)
    path_to_file = pjoin("D:\\", "DataAnaly", "python", "BAX422","midterm",
                         "sponsored", filename)
    with open(path_to_file, 'r',encoding="utf8") as file:
        text = file.read()
        soup = BeautifulSoup(text, 'html.parser') 
        
        
        #Select seller_name
        seller_name = soup.find("span", attrs={"class": "mbg-nw"}).text if bool(soup.find("span", attrs={"class": "mbg-nw"}))== True else None

        
        #Select seller score
        seller_score = soup.find("span",attrs={"class": "mbg-l"}) .findChild("a").text if bool(soup.find("span",attrs={"class": "mbg-l"}))== True else None
                                  
        #Select item price
        if bool(soup.find("span",attrs={"id": "convbidPrice"}))== True:
            item_price = re.sub(r'\([^)]*\)','',soup.find("span",attrs={"id": "convbidPrice"}).get_text())
        elif bool(soup.find("span",attrs={"id": "prcIsum"}))== True:
            item_price = soup.find("span",attrs={"id": "prcIsum"}).text
        elif bool(soup.find("span",attrs={"id": "priceIsum _bidPrice"})) ==True:
            item_price = soup.find("span",attrs={"id": "priceIsum _bidPrice"}).text
        elif bool(soup.find("span",attrs={"id": "mm-saleDscPrc"}))== True:
            item_price = soup.find("span",attrs={"id": "mm-saleDscPrc"}).text
        elif soup.find("span",attrs={"class": "ms-si ms-id"}) is not None:
            item_price = seleniumgetprice(path_to_file)
        else:
            item_price is np.nan  
        
        #Select number of items sold
        item_sold =soup.find("a",attrs={"class": "vi-txt-underline"}).text if bool(soup.find("a",attrs={"class": "vi-txt-underline"}))== True else None
        
        
        #Select best offer available
        bestoffer ="True" if bool(soup.find("a", attrs = {"id":"boBtn_btn"}))== True else "False"
        
        #Select title
        itemtitle =  soup.find("h1",attrs={"id":"itemTitle"}).text if bool(soup.find("h1",attrs={"id":"itemTitle"}))== True else None
    
        #Select returns allowed
        return_allow = soup.find("td",attrs={"class": "rpTblColm"}).find('div').text.strip() if bool(soup.find("td",attrs={"class": "rpTblColm"}))== True else None
                                   
        #select shipping price
        if bool(soup.find("a",attrs={"class": "si-pd sh-nwr"})) is True:
            shipprice = np.nan
            shipprice_calculate = "1"
        elif bool(soup.find("span",attrs={"id": "fshippingCost"})) is True:
            if len(re.findall(r'[\$](\d+(?:\.\d{2}?))',soup.find("span",attrs={"id": "fshippingCost"}).text))>1:
                shipprice = (re.findall(r'[\$](\d+(?:\.\d{2}?))',soup.find("span",attrs={"id": "fshippingCost"}).text))[1]
                shipprice_calculate = "0"
            else: 
                shipprice = soup.find("span",attrs={"id": "fshippingCost"}).text
                shipprice_calculate = "0"
        elif bool(soup.find("span",attrs={"class": "vi-fnf-ship-txt fnfgreen"}))is True:
            shipprice = "Free"
            shipprice_calculate = "0"
        else:
            shipprice = np.nan
            shipprice_calculate = "0"
        
        #Select condition (e.g., used, new, like new, seller refurbished)
        condition = soup.find("div",attrs={"id": "vi-itm-cond"}).text if bool(soup.find("div",attrs={"id": "vi-itm-cond"}))== True else None
 
        # store the info
        df = df.append({'item_id':i , 
                        'item_title': itemtitle,
                        'seller_name': seller_name,
                        'seller_score': seller_score,
                        'item_price': item_price,
                        'item_sold': item_sold,
                        'best_offer': bestoffer,
                        'return_policy': return_allow,
                        'shipping_price': shipprice,
                        'shipping_price_calculate': shipprice_calculate,
                        'condition': condition,
                        'sponsor': "sponsored"}, ignore_index=True)


In [85]:
# write all the information of non-sponsored item in dataframe "df"

for i in nonsponsored_id:
    filename = "%s.htm" % (i)
    path_to_file = pjoin("D:\\", "DataAnaly", "python", "BAX422","midterm",
                         "non-sponsored", filename)
    with open(path_to_file, 'r',encoding="utf8") as file:
        text = file.read()
        soup = BeautifulSoup(text, 'html.parser') 
        
        
        #Select seller_name
        seller_name = soup.find("span", attrs={"class": "mbg-nw"}).text if bool(soup.find("span", attrs={"class": "mbg-nw"}))== True else None

        
        #Select seller score
        seller_score = soup.find("span",attrs={"class": "mbg-l"}) .findChild("a").text if bool(soup.find("span",attrs={"class": "mbg-l"}))== True else None
                                  
        #Select item price
          
        if bool(soup.find("span",attrs={"id": "convbidPrice"}))== True:
            item_price = re.sub(r'\([^)]*\)','',soup.find("span",attrs={"id": "convbidPrice"}).get_text())
        elif bool(soup.find("span",attrs={"id": "prcIsum"}))== True:
            item_price = soup.find("span",attrs={"id": "prcIsum"}).text
        elif bool(soup.find("span",attrs={"id": "priceIsum _bidPrice"})) ==True:
            item_price = soup.find("span",attrs={"id": "priceIsum _bidPrice"}).text
        elif bool(soup.find("span",attrs={"id": "mm-saleDscPrc"}))== True:
            item_price = soup.find("span",attrs={"id": "mm-saleDscPrc"}).text
        elif soup.find("span",attrs={"class": "ms-si ms-id"}) is not None:
            item_price = seleniumgetprice(path_to_file)
        else:
            item_price is np.nan  
        
        
        #Select number of items sold
        item_sold =soup.find("a",attrs={"class": "vi-txt-underline"}).text if bool(soup.find("a",attrs={"class": "vi-txt-underline"}))== True else None
        
        
        #Select best offer available
        bestoffer ="True" if bool(soup.find("a", attrs = {"id":"boBtn_btn"}))== True else "False"
        
        #Select title
        itemtitle =  soup.find("h1",attrs={"id":"itemTitle"}).text if bool(soup.find("h1",attrs={"id":"itemTitle"}))== True else None
        
        #Select returns allowed
        return_allow = soup.find("td",attrs={"class": "rpTblColm"}).find('div').text.strip() if bool(soup.find("td",attrs={"class": "rpTblColm"}))== True else None
                                   
        #select shipping price

        if bool(soup.find("a",attrs={"class": "si-pd sh-nwr"})) is True:
            shipprice = np.nan
            shipprice_calculate = "1"
        elif bool(soup.find("span",attrs={"id": "fshippingCost"})) is True:
            if len(re.findall(r'[\$](\d+(?:\.\d{2}?))',soup.find("span",attrs={"id": "fshippingCost"}).text))>1:
                shipprice = (re.findall(r'[\$](\d+(?:\.\d{2}?))',soup.find("span",attrs={"id": "fshippingCost"}).text))[1]
                shipprice_calculate = "0"
            else: 
                shipprice = soup.find("span",attrs={"id": "fshippingCost"}).text
                shipprice_calculate = "0"
        elif bool(soup.find("span",attrs={"class": "vi-fnf-ship-txt fnfgreen"}))is True:
            shipprice = "Free"
            shipprice_calculate = "0"
        else:
            shipprice = np.nan
            shipprice_calculate = "0"
        
        #Select condition (e.g., used, new, like new, seller refurbished)
        condition = soup.find("div",attrs={"id": "vi-itm-cond"}).text if bool(soup.find("div",attrs={"id": "vi-itm-cond"}))== True else None
 
        # store the info
        df = df.append({'item_id':i , 
                        'item_title': itemtitle,
                        'seller_name': seller_name,
                        'seller_score': seller_score,
                        'item_price': item_price,
                        'item_sold': item_sold,
                        'best_offer': bestoffer,
                        'return_policy': return_allow,
                        'shipping_price': shipprice,
                        'shipping_price_calculate': shipprice_calculate,
                        'condition': condition,
                        'sponsor': "non-sponsored"}, ignore_index=True)

Clean Data

In [103]:
dfclean = df.copy()

In [104]:
# clean "item_title"
dfclean["item_title"] = dfclean["item_title"].replace({"Details about ": ""},regex=True)

In [124]:
# clean "item_price"
dfclean["item_price"] = dfclean["item_price"].replace({"\$": ""},regex=True).replace({"US":""},regex=True).replace({"\/ea":""},regex=True).replace({",":""},regex=True)

In [106]:
# clean "shipping_price"
dfclean["shipping_price"] = (np.where(dfclean["shipping_price"] == "FREE" ,
                                     "0.00",
                                      np.where(dfclean["shipping_price"] == "Free",
                                               "0.00",
                                               dfclean["shipping_price"].replace({"\$":""},regex=True).replace({"\n":""},regex=True))
                                     )
)


In [108]:
# clean "item_sold"
dfclean["item_sold"] = dfclean["item_sold"].replace({' sold':''},regex=True).replace({',':''},regex=True)

In [110]:
# clean "return_policy"
dfclean["return_policy"] = (np.where(dfclean["return_policy"] =="No Return",
                                     "No",
                                     np.where(dfclean["return_policy"] .isnull(),
                                             "No",
                                             "Yes")
                                    ))

**e)Dump information in the database**  

Use your code script to connect to SQL (either MySQL, MariaDB, or SQLite. Do NOT use SQL GUI or command terminal). Create a database and name it "eBay". Save the information of items in (d)  into a single table named "eBay_items" (You are allowed to use only one table). This table should contain both sponsored and non-sponsored information and have a column that specifies which item is sponsored/non-sponsored.  If an item misses ANY of the information in (d), you should insert that missing value as NULL into the table. Convert any price (item price and shipping price) into a "dollar-cent" format (e.g., convert $12.34 into 1234 and $12 into 1200. Make sure the two least significant digits are cents. If an item does not include cents in the price, insert zeros.) and insert the price as INT into the table.

In [112]:
import mysql.connector
from mysql.connector import Error
from mysql.connector import errorcode

In [113]:
# 1) build connection
myconnect = mysql.connector.connect(
        host='localhost',
        user='root',
        connect_timeout=5,
        password = '')
print('Connect successfully' if myconnect else 'Disconnected')

Connect successfully


In [114]:
# 2) creates a database named "ebay
def build_db(cursor,dbname):
    cursor.execute('DROP DATABASE IF EXISTS %s' % (dbname))
    cursor.execute('CREATE DATABASE %s' % (dbname))
    print('Build %s successfully' % (dbname))
    return

mycursor = myconnect.cursor()

build_db(mycursor, 'eBay')

Build eBay successfully


In [125]:
# write dataframe into sql table
from sqlalchemy import create_engine
import pymysql

tableName   = "ebay_item"
dataFrame   = pd.DataFrame(data=dfclean)           

 
#build mysql connection
sqlEngine       = create_engine('mysql+pymysql://root:@127.0.0.1/ebay', pool_recycle=3600)
dbConnection    = sqlEngine.connect()

 

try:

    frame   = dataFrame.to_sql(tableName, dbConnection, if_exists='replace');

except ValueError as vx:

    print(vx)

except Exception as ex:   

    print(ex)

else:

    print("Table %s created successfully."%tableName);   

finally:

    dbConnection.close()

Table ebay_item created successfully.


In [127]:
myconnect = mysql.connector.connect(
        host='localhost',
        user='root',
        connect_timeout=5,
        database = "eBay",
        password = '')

mycursor = myconnect.cursor()


# change the data type to the correct one
mycursor.execute("ALTER TABLE ebay.ebay_item MODIFY item_id char(12)")
mycursor.execute("ALTER TABLE ebay.ebay_item MODIFY item_title varchar(255)")
mycursor.execute("ALTER TABLE ebay.ebay_item MODIFY seller_name varchar(32)")
mycursor.execute("ALTER TABLE ebay.ebay_item MODIFY seller_score integer")
mycursor.execute("ALTER TABLE ebay.ebay_item MODIFY item_price float")
mycursor.execute("ALTER TABLE ebay.ebay_item MODIFY item_sold int8")
mycursor.execute("ALTER TABLE ebay.ebay_item MODIFY return_policy varchar(8)")
mycursor.execute("ALTER TABLE ebay.ebay_item MODIFY shipping_price float")
mycursor.execute("ALTER TABLE ebay.ebay_item MODIFY best_offer varchar(8)")
mycursor.execute("ALTER TABLE ebay.ebay_item MODIFY `condition` varchar(64)")
mycursor.execute("ALTER TABLE ebay.ebay_item MODIFY return_policy varchar(8)")
mycursor.execute("ALTER TABLE ebay.ebay_item MODIFY sponsor varchar(16)")


In [128]:
# recalculate the price into dollar cent and change the datatype of price to integer
sql = ('UPDATE ebay.ebay_item  '
        'SET item_price = item_price*100, '
        '    shipping_price = shipping_price*100 ')
mycursor.execute(sql)

mycursor.execute("ALTER TABLE ebay.ebay_item MODIFY item_price integer")
mycursor.execute("ALTER TABLE ebay.ebay_item MODIFY shipping_price integer")

myconnect.commit()
mycursor.close()
myconnect.close()

f) Use your code script (and NOT SQL GUI or command terminal) to run summary stats on each item. Print to the screen the mean, min, max, and mean for each column, grouped by "sponsor/non-sponsor" and "condition" (group by at the same time, not separately). For binary categorical columns, use 0-1 conversion. For e.g., for the "returns allowed" convert YES to 1 and NO to 0 and then calculate the stats. If it is NOT a numerical/binary categorical column, print to the screen the count of each category level. You will need to ignore NULL values in your statistic calculations. 

In [139]:
myconnect = mysql.connector.connect(
        host='localhost',
        user='root',
        connect_timeout=5,
        database = "eBay",
        password = '')

mycursor = myconnect.cursor()


# convert "best_offer" and "return_policy" （Yes/True - 1, No/False - 0)
sql1 = ('UPDATE ebay.ebay_item SET best_offer = 0 WHERE best_offer = "False"')
mycursor.execute(sql1)

sql2 = ('UPDATE ebay.ebay_item SET best_offer = 1 WHERE best_offer = "True" ')
mycursor.execute(sql2)
       

sql3= ('UPDATE ebay.ebay_item SET return_policy = 1 WHERE return_policy = "Yes"')
mycursor.execute(sql3)       


sql4= ('UPDATE ebay.ebay_item SET return_policy = 0 WHERE return_policy = "No"')
mycursor.execute(sql4)       

mycursor.execute("ALTER TABLE ebay.eBay_item MODIFY best_offer BINARY(16)")
mycursor.execute("ALTER TABLE ebay.eBay_item MODIFY return_policy BINARY(16)")

myconnect.commit()
mycursor.close()
myconnect.close()

In [140]:
myconnect = mysql.connector.connect(
        host='localhost',
        user='root',
        connect_timeout=5,
        database = "eBay",
        password = '')

mycursor = myconnect.cursor()

mycursor.execute("""
        select sponsor,
               `condition`,
               avg(item_price),
               max(item_price),
               min(item_price),
                avg(item_sold),
               min(item_sold),
               max(item_sold),
               sum(best_offer) as count_best_offer,
               sum(return_policy) as count_return_allowed,
               avg(shipping_price),
               max(shipping_price),
               min(shipping_price)
        from ebay.ebay_item
        group by sponsor, `condition`
        """
        )

result = mycursor.fetchall()



query_df =  pd.DataFrame(data=result, columns=["sponsor","condition","mean_itemprice",
                           "max_item_price","min_item_price","mean_item_sold"
                           ,"min_item_sold","max_item_sold"
                           ,"count_best_offer","count_return_allow","mean_shipping_price"
                                 ,"max_shipping_price", "min_shipping_price"])


In [141]:
query_df

,sponsor,condition,mean_itemprice,max_item_price,min_item_price,mean_item_sold,min_item_sold,max_item_sold,count_best_offer,count_return_allow,mean_shipping_price,max_shipping_price,min_shipping_price
0,sponsored,Seller refurbished,23459.6000,25300,19999,871.5000,4,1739,2.0,3.0,0.0000,0,0
1,sponsored,New,32458.8085,57999,22399,33.5714,1,315,9.0,36.0,646.5897,6798,0
2,sponsored,Used,25550.3333,55351,15000,356.4000,7,1690,35.0,32.0,1350.7381,5266,0
3,sponsored,Open box,32459.0000,52995,23800,5.0000,5,5,3.0,2.0,0.0000,0,0
4,sponsored,Manufacturer refurbished,26133.0000,33900,19999,3.0000,3,3,2.0,1.0,0.0000,0,0
5,non-sponsored,New,31638.3678,126999,17499,79.3924,1,1896,95.0,167.0,371.4055,4800,0
6,non-sponsored,Used,22924.4588,50000,3900,9.3333,1,58,196.0,235.0,585.2670,4660,0
7,non-sponsored,Open box,27470.0000,67500,16000,4.0000,3,5,27.0,12.0,921.1304,5700,0
8,non-sponsored,Seller refurbished,23285.5366,36995,13240,125.4167,1,1224,12.0,23.0,321.3333,2999,0
9,non-sponsored,Manufacturer refurbished,22799.8333,30000,19000,5.0000,5,5,4.0,2.0,0.0000,0,0
